# Llama2 학습을 위한 KorQuad 데이터셋 정제

# KorQuAD v2.0 (dev) 데이터 다운로드
https://korquad.github.io/

In [2]:
import json

dev_data = []
for i in range(0, 5):
    with open(f'./dataset/korquad2.1_dev_0{i}.json', 'r') as f:
        data = json.load(f)
        data = [item for topic in data['data'] for item in topic['qas']]
        dev_data.append(data)
        

In [3]:
dev_data = sum(dev_data, [])

In [4]:
print(dev_data[0])

{'answer': {'text': '<td>332cm</td>', 'html_answer_start': 45900, 'html_answer_text': '<td>332cm</td>', 'answer_start': 10698}, 'question': '파스칼레 소틸레의 스파이크 높이는 몇 cm인가?', 'id': '38824'}


In [5]:
len(dev_data)

10165

# KorQuad 데이터셋을 Llama2 학습을 위한 Prompt 형태로 정제

In [6]:
refined_dict = {}
for entry in dev_data:
    question = entry['question']
    answers = entry['answer']['text']
    refined_dict[question] = answers

In [7]:
print(len(refined_dict))
print(refined_dict['2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 2015-16년에 어느 팀 소속이었나?'])

10159
MKS 벵진


In [8]:
final_prompt_list = []
for idx, (question, answer) in enumerate(refined_dict.items()):
  prompt = f"## Question: {question} ## Answer: {answer}"
  prompt_dict = {}
  prompt_dict['text'] = prompt
  prompt_dict['question'] = question
  prompt_dict['answer'] = answer

  final_prompt_list.append(prompt_dict)
  
print(final_prompt_list[0])

{'text': '## Question: 파스칼레 소틸레의 스파이크 높이는 몇 cm인가? ## Answer: <td>332cm</td>', 'question': '파스칼레 소틸레의 스파이크 높이는 몇 cm인가?', 'answer': '<td>332cm</td>'}


In [9]:
print('데이터 개수 :', len(final_prompt_list))

데이터 개수 : 10159


In [10]:
num_items = 20

for idx, item in enumerate(final_prompt_list):
    if num_items < idx:
        break
    print(item)

{'text': '## Question: 파스칼레 소틸레의 스파이크 높이는 몇 cm인가? ## Answer: <td>332cm</td>', 'question': '파스칼레 소틸레의 스파이크 높이는 몇 cm인가?', 'answer': '<td>332cm</td>'}
{'text': '## Question: 2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 2015-16년에 어느 팀 소속이었나? ## Answer: MKS 벵진', 'question': '2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 2015-16년에 어느 팀 소속이었나?', 'answer': 'MKS 벵진'}
{'text': '## Question: 혈액 검사에서 나트륨의 정상범위는 최소 136mmol/L에서 최대 몇 mmol/L까지 일까? ## Answer: <td>145</td>', 'question': '혈액 검사에서 나트륨의 정상범위는 최소 136mmol/L에서 최대 몇 mmol/L까지 일까?', 'answer': '<td>145</td>'}
{'text': '## Question: 안토닌 드보르자크의 작품 중 18개의 가곡으로 이루어진 노래는? ## Answer: 사이프러스', 'question': '안토닌 드보르자크의 작품 중 18개의 가곡으로 이루어진 노래는?', 'answer': '사이프러스'}
{'text': '## Question: 에드먼드 핼리가 달의 운동이 빨라지고 있다고 주장한 년도는 언제일까? ## Answer: 1695년', 'question': '에드먼드 핼리가 달의 운동이 빨라지고 있다고 주장한 년도는 언제일까?', 'answer': '1695년'}
{'text': '## Question: 지구의 속도를 낮추고 달의 궤도를 올라가게 하는 것은 무엇일까? ## Answer: 중력 토크', 'question': '지구의 속도를 낮추고 달의 궤도를 올라가게 하는 것은 무엇일까?', 'answer': '중력 토크'}


## JSON 파일로 저장 및 다운로드

In [11]:
with open('./dataset/korquad2_dev_train.json', 'w', encoding='utf-8') as f:
    f.write("[\n")
    for i, item in enumerate(final_prompt_list):
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(json_str)
        if i < len(final_prompt_list) - 1:  # 마지막 원소가 아니라면 콤마를 추가
            f.write(",\n")
        else:  # 마지막 원소라면 콤마를 추가하지 않음
            f.write("\n")
    f.write("]\n")